### Performing Paired Montecarlo Iterations for different functional units and different methods 

Pierre Jouannais 2023

That's the only way I have found to keep everything paired accross the iterations.

In [20]:
import bw2data
import bw2io
from bw2data.parameters import *
import brightway2 as bw
import time
import numpy as np
import pandas as pd

In [7]:

# Load whatever project in which you have a version of ecoinvent

bw.projects.set_current('AH_38_naturalgas_2_0711') 



# Loading Ecoinvent
Ecoinvent = bw.Database('ecoinvent 3.8 conseq')


In [8]:
# Choose the background activities (here random)

act1=Ecoinvent.random()
act2=Ecoinvent.random()
act3=Ecoinvent.random()


In [39]:

# 3 background activities in a list 
list_FU=[{act1:1},{act2:1},{act3:1}]

# 5 methods
list_meth =[('ReCiPe Midpoint (I)', 'climate change', 'GWP20'), 
            ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf'),
            ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf'),
            ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf'),
            ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')] # We use the same twice to check that the code works
 


 1) Collect the necessary characterization matrixes in a dictionnary 

In [40]:

    

# Create empty dictionnary to collect characterization matrixes
C_matrixes ={}

# Make a LCA. It's only to load a LCA object. We do not care about the result.
Lca=bw.LCA(list_FU[1],('ReCiPe Midpoint (I)', 'climate change', 'GWP20'))
Lca.lci()
Lca.lcia()

# Use switch_method and collect the characterization matrix for every change
for meth in list_meth:
    Lca.switch_method(meth)
    #print(meth,"##############")
    #print(Lca.characterization_matrix)
    C_matrixes[meth]=Lca.characterization_matrix



2 ) Calculate 


In [41]:


time1=time.time()  

# 5 Montecarlo iterations
mciteration = 5

# Initalize the list of results ( does not have to be this structure)

listarray_mc_sample = [np.array([[0]*len(list_FU)  ]*mciteration,dtype="float32") for meth in range(len(list_meth))]

# Create Montecarlo object
mc=bw.MonteCarloLCA(list_FU[2],('ReCiPe Midpoint (I)', 'climate change', 'GWP20'))

for it in range(mciteration):
    
    #print("ok1",it)
    next(mc)
    #print("ok2",it)


    for i in range(0,len(list_FU)):
       
        mc.redo_lcia(list_FU[i])  # redo with new FU
        
        index_array_method=-1

        for m in list_meth:
        
            print("ok3",m)

            index_array_method+=1
        
            
            listarray_mc_sample[index_array_method][it,i]=(C_matrixes[m]*mc.inventory).sum() # This calculates the LCIA
            

time2=time.time()
            
tot_time = time2-time1 

ok3 ('ReCiPe Midpoint (I)', 'climate change', 'GWP20')
ok3 ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf')
ok3 ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf')
ok3 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')
ok3 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')
ok3 ('ReCiPe Midpoint (I)', 'climate change', 'GWP20')
ok3 ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf')
ok3 ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf')
ok3 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')
ok3 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')
ok3 ('ReCiPe Midpoint (I)', 'climate change', 'GWP20')
ok3 ('ReCiPe Midpoint (H)', 'human toxicity', 'HTPinf')
ok3 ('ReCiPe Midpoint (H)', 'freshwater ecotoxicity', 'FETPinf')
ok3 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')
ok3 ('ReCiPe Midpoint (H)', 'terrestrial ecotoxicity', 'TETPinf')
ok3 ('ReCiPe Midpoint (I)', 'climate change', 'GWP20')
ok3 ('ReCiPe Midpoint

I just put the results into a dictionnary

In [46]:
dict_dataframes = {str(i) + "_" + list_meth[i][1] :pd.DataFrame(listarray_mc_sample[i]) for i in range(len(list_meth))}

In [47]:
dict_dataframes

{'0_climate change':            0             1         2
 0  16.562298  0.000000e+00 -0.395672
 1  15.450603 -6.042763e-07 -0.460029
 2  15.891187 -5.047495e-08 -0.330496
 3  26.291393 -2.161668e-07 -0.340411
 4  18.694324 -2.672071e-07 -0.341901,
 '1_human toxicity':            0             1         2
 0   5.332920  0.000000e+00 -0.488250
 1  11.189725  4.759254e-07 -1.157514
 2   7.202059  2.960252e-06 -0.488480
 3   9.587682  5.205783e-07 -0.731883
 4   5.628643  7.762278e-07 -0.829702,
 '2_freshwater ecotoxicity':           0             1         2
 0  0.405248  0.000000e+00 -0.091129
 1  0.310291  3.302534e-07 -0.250278
 2  0.570379  2.073357e-07 -0.082415
 3  0.635209  1.990173e-07 -0.152199
 4  0.358460  2.711457e-08 -0.146770,
 '3_terrestrial ecotoxicity':           0             1         2
 0  0.146423  0.000000e+00 -0.000016
 1  0.019504 -1.125576e-08 -0.000007
 2  0.034271  3.531594e-09 -0.000009
 3  0.062775 -4.611116e-09 -0.000007
 4  0.036978 -1.088558e-08 -0.000004,

In [53]:
dict_dataframes['0_climate change']

,0,1,2
0,16.562298,0.000000e+00,-0.395672
1,15.450603,-6.042763e-07,-0.460029
2,15.891187,-5.047495e-08,-0.330496
3,26.291393,-2.161668e-07,-0.340411
4,18.694324,-2.672071e-07,-0.341901


Check if the iterations give the same results for the two same methods,  to check that all iterations are paired.

In [50]:
dict_dataframes['3_terrestrial ecotoxicity']

,0,1,2
0,0.146423,0.000000e+00,-0.000016
1,0.019504,-1.125576e-08,-0.000007
2,0.034271,3.531594e-09,-0.000009
3,0.062775,-4.611116e-09,-0.000007
4,0.036978,-1.088558e-08,-0.000004


In [51]:
dict_dataframes['4_terrestrial ecotoxicity']  

,0,1,2
0,0.146423,0.000000e+00,-0.000016
1,0.019504,-1.125576e-08,-0.000007
2,0.034271,3.531594e-09,-0.000009
3,0.062775,-4.611116e-09,-0.000007
4,0.036978,-1.088558e-08,-0.000004


Ok